# Exploration of the air quality data

In [3]:
# Install the necessary packages directly within Jupyter
!pip install sqlalchemy pymysql pandas matplotlib seaborn


  Using cached SQLAlchemy-2.0.36-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached greenlet-3.1.1-cp311-cp311-win_amd64.whl.metadata (3.9 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
Using cached SQLAlchemy-2.0.36-cp311-cp311-win_amd64.whl (2.1 MB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.0 MB 2.6 

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Database connection parameters
host = 'air-quality-db.c6jid8c0mhxf.us-east-1.rds.amazonaws.com'
port = 3306  # The default port for MySQL
user = 'admin'  # Replace with your actual username
password = 'aqdata.09'  # Consider managing this securely as mentioned
database = 'air_quality_2020'

# Create a connection string using pymysql as the driver for MySQL
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'

# Create a database engine
engine = create_engine(connection_string)

# Query each year's table and perform analysis
for year in range(2020, 2024):
    table_name = f'air_quality_{year}'
    query = f'SELECT * FROM {table_name}'
    df = pd.read_sql(query, engine)

    # Display the first few rows of the dataframe
    print(f"Data for {year}:")
    print(df.head())

    # Generate a histogram for a column, for example, PM2.5 readings
    plt.figure(figsize=(10, 6))
    sns.histplot(df['PM2_5'], kde=True)  # Replace 'PM2_5' with the actual column name if different
    plt.title(f'Distribution of PM2.5 in {year}')
    plt.show()

    # Calculate and print correlation matrix
    print("Correlation matrix:")
    print(df.corr())

    # Optionally, create more visualizations or statistical summaries


OperationalError: (pymysql.err.OperationalError) (1049, "Unknown database 'air_quality_2020'")
(Background on this error at: https://sqlalche.me/e/20/e3q8)